In [ ]:
import numpy as np
import pandas as pd

np.random.seed(7)

# ============================================================
# 1. Time index: 600 days of hourly data
# ============================================================
D = 600
H = 24
T = D * H

idx = pd.date_range(start="2023-01-01 00:00:00", periods=T, freq="H")
hours = idx.hour.values          # 0..23
dow = idx.dayofweek.values       # 0=Mon .. 6=Sun

# ============================================================
# 2. Workload generation (0–10, office worker pattern)
#    - Higher on weekdays, daytime
#    - Lower at night & weekends
# ============================================================

# Diurnal pattern: bump around 13:00 (1 pm)
center = 13.0
diurnal = 0.5 * (1 + np.cos(2 * np.pi * (hours - center) / 24.0))
diurnal = 1 - diurnal  # low at night, high during day

# Weekday vs weekend factor
weekday_factor = np.where(dow < 5, 1.0, 0.5)  # weekends ~50% load

# Deterministic mean workload over time
w0, A = 2.0, 6.0
mu_t = w0 + A * diurnal * weekday_factor  # between ~2 and ~8

# AR(1) noise around mu_t
phi_w, sigma_w = 0.85, 0.8
eps_w = np.random.normal(0, sigma_w, size=T)

w_raw = np.zeros(T)
w_raw[0] = mu_t[0] + eps_w[0]
for t in range(1, T):
    w_raw[t] = mu_t[t] + phi_w * (w_raw[t-1] - mu_t[t-1]) + eps_w[t]

workload = np.clip(w_raw, 0, 10)

# ============================================================
# 3. Stress generation (0–10), correlated with workload
# ============================================================

phi_b, sigma_b = 0.90, 0.4   # AR(1) baseline
alpha = 0.8                  # sensitivity to workload
sigma_eta = 0.5              # idiosyncratic stress noise

base = np.zeros(T)
eps_b = np.random.normal(0, sigma_b, size=T)
base[0] = eps_b[0]
for t in range(1, T):
    base[t] = phi_b * base[t-1] + eps_b[t]

eta = np.random.normal(0, sigma_eta, size=T)

stress_raw = base + alpha * workload + eta + 2.0
stress = np.clip(stress_raw, 0, 10)

# ============================================================
# 4. HRV generation (RMSSD, ms), penalized by stress
#    - Generated in log-space (log-normal)
# ============================================================

mu_log_hrv = np.log(40.0)   # baseline HRV ~40 ms
phi_hrv = 0.90
gamma = 0.06                # stress penalty
sigma_eps = 0.12
s_bar = 5.0

eps = np.random.normal(0, sigma_eps, size=T)
x = np.zeros(T)

# first step
x[0] = mu_log_hrv - gamma * (stress[0] - s_bar) + eps[0]

for t in range(1, T):
    x[t] = (mu_log_hrv
            + phi_hrv * (x[t-1] - mu_log_hrv)
            - gamma * (stress[t] - s_bar)
            + eps[t])

hrv = np.exp(x)              # back to ms
hrv = np.clip(hrv, 10, 120)  # keep in a physiological-ish range

# ============================================================
# 5. NEW migraine probability model (main change)
#
#   - Simple normalized risk from workload, stress, low HRV
#   - Then a calibrated logistic to get nice 0.2 / 0.3 / 0.7 values
# ============================================================

# 5.1 Normalize features
w = workload
s = stress
h = hrv

# workload, stress in [0,10] -> [0,1]
w_norm = w / 10.0
s_norm = s / 10.0

# HRV: higher HRV = better, so invert it
HRV_MIN = 20.0
HRV_MAX = 100.0

h_clipped = np.clip(h, HRV_MIN, HRV_MAX)
h_norm = (HRV_MAX - h_clipped) / (HRV_MAX - HRV_MIN)
# h_norm = 0 → very good HRV, h_norm = 1 → very poor HRV

# 5.2 Weighted linear risk (0 ~ calm, 1 ~ very bad)
risk_linear = 0.5 * s_norm + 0.3 * w_norm + 0.2 * h_norm

# 5.3 Calibrated logistic mapping
#     Choose a,b so that:
#       risk_linear = 0.2 -> p ≈ 0.2
#       risk_linear = 0.5 -> p ≈ 0.5
#       risk_linear = 0.8 -> p ≈ 0.8
#     This gives:
#       a ≈ -2.31,  b ≈ 4.62
a = -2.31
b =  4.62

lin = a + b * risk_linear
p_migraine_next_hour = 1.0 / (1.0 + np.exp(-lin))

# ============================================================
# 6. Quick diagnostics: how does the distribution look?
# ============================================================

def describe_probs(p):
    qs = [1, 5, 25, 50, 75, 95, 99]
    print("p_migraine_next_hour min/max:",
          float(np.min(p)), float(np.max(p)))
    for q in qs:
        print(f"  {q:2d}th percentile: {np.percentile(p, q):.3f}")
    print("  mean probability:", float(np.mean(p)))
    print("  fraction > 0.5:", float(np.mean(p > 0.5)))
    print("  fraction between 0.2 and 0.8:",
          float(np.mean((p >= 0.2) & (p <= 0.8))))

print("=== Migraine probability diagnostics ===")
describe_probs(p_migraine_next_hour)

# Now you should see plenty of values around 0.2, 0.3, 0.7, etc.

# ============================================================
# 7. Assemble DataFrame and save to CSV
# ============================================================

df = pd.DataFrame({
    "timestamp": idx,
    "workload_0_10": np.round(workload, 2),
    "stress_0_10": np.round(stress, 2),
    "hrv_rmssd_ms": np.round(hrv, 2),
    "migraine_prob_next_hour": np.round(p_migraine_next_hour, 4),
})

df.set_index("timestamp", inplace=True)

out_path = "synthetic_wsht_weather_migraine_prob_600days_hourly_FIXED.csv"
df.to_csv(out_path)
print(f"\nSaved CSV to: {out_path}")
print("First few rows:")
print(df.head())


=== Migraine probability diagnostics ===
p_migraine_next_hour min/max: 0.09029814470291984 0.9097018552970803
   1th percentile: 0.135
   5th percentile: 0.206
  25th percentile: 0.416
  50th percentile: 0.585
  75th percentile: 0.741
  95th percentile: 0.872
  99th percentile: 0.904
  mean probability: 0.5696168310288757
  fraction > 0.5: 0.6354861111111111
  fraction between 0.2 and 0.8: 0.8011111111111111

Saved CSV to: synthetic_wsht_migraine_prob_600days_hourly_CALIBRATED.csv
First few rows:
                     workload_0_10  stress_0_10  hrv_rmssd_ms  \
timestamp                                                       
2023-01-01 00:00:00           6.30         7.31         38.47   
2023-01-01 01:00:00           5.78         6.16         36.89   
2023-01-01 02:00:00           5.64         6.06         33.42   
2023-01-01 03:00:00           5.71         5.96         32.00   
2023-01-01 04:00:00           4.70         3.62         34.05   

                     migraine_prob_next_ho

C:\Users\peace\AppData\Local\Temp\ipykernel_52376\2959377235.py:13: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  idx = pd.date_range(start="2023-01-01 00:00:00", periods=T, freq="H")
